In [1]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')

import pandas as pd
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

import os
from datetime import datetime

from ff_app.execution import run_data_pull, update_google_sheet
from ff_app import data_prep

05:20:23 INFO:NumExpr defaulting to 8 threads.


In [2]:
week = 13

short_df, combined_df = run_data_pull(week=week)
# short_df, combined_df, all_df = run_data_pull(week=week, return_all_games=True)
print(short_df.shape)
print(combined_df.shape)

05:20:23 DEBUG:Starting new HTTP connection (1): www.espn.com:80
05:20:23 DEBUG:http://www.espn.com:80 "GET /college-football/scoreboard/_/group/80/year/2021/seasontype/2/week/13 HTTP/1.1" 301 241
05:20:23 DEBUG:Starting new HTTPS connection (1): www.espn.com:443
05:20:24 DEBUG:https://www.espn.com:443 "GET /college-football/scoreboard/_/group/80/year/2021/seasontype/2/week/13 HTTP/1.1" 200 None
05:20:26 INFO:Pulled 65 total games for week 13 in 2021
05:20:26 INFO:65 games have odds available
05:20:26 INFO:Picks sheet data has shape (65, 9)
05:20:26 INFO:Game list data has shape (65, 27)
05:20:26 INFO:Saving game data to disk at '/Users/hoke/dev/football/game_lists/2021/week13.csv'


(65, 9)
(65, 27)


In [3]:
short_df.head()

,Week,Datetime,Mandatory,Favorite,Location,Underdog,Spread,Total,Implied Score
0,13,11/23 07:00 PM (Tue),,BALL,vs,BUFF,-6.5,59.5,BALL 33 - BUFF 26
1,13,11/23 07:00 PM (Tue),,WMU,@,NIU,-6.0,60.5,WMU 33 - NIU 27
2,13,11/25 03:30 PM (Thu),,FRES,@,SJSU,-7.5,52.5,FRES 30 - SJSU 22
3,13,11/25 07:30 PM (Thu),Y,MSST,vs,MISS,-1.5,62.0,MSST 31 - MISS 30
4,13,11/26 12:00 PM (Fri),,BSU,@,SDSU,-2.5,44.5,BSU 23 - SDSU 21


In [4]:
short_df[(short_df['Favorite'] == 'CLEM') | (short_df['Underdog'] == 'CLEM')]

,Week,Datetime,Mandatory,Favorite,Location,Underdog,Spread,Total,Implied Score
59,13,11/27 07:30 PM (Sat),,CLEM,@,SC,-11.5,43.0,CLEM 27 - SC 15


In [5]:
short_df[short_df['Mandatory'] == 'Y']

,Week,Datetime,Mandatory,Favorite,Location,Underdog,Spread,Total,Implied Score
3,13,11/25 07:30 PM (Thu),Y,MSST,vs,MISS,-1.5,62.0,MSST 31 - MISS 30
21,13,11/27 12:00 PM (Sat),Y,OSU,@,MICH,-8.0,64.5,OSU 36 - MICH 28
56,13,11/27 07:30 PM (Sat),Y,OKST,vs,OU,-4.0,49.5,OKST 26 - OU 22


In [9]:
team_list = ['ORE']

mandatory_picks = short_df[(short_df['Favorite'].isin(team_list)) |
                           (short_df['Underdog'].isin(team_list))]
mandatory_picks

,Week,Datetime,Mandatory,Favorite,Location,Underdog,Spread,Total,Implied Score
42,13,11/27 03:30 PM (Sat),Y,ORE,vs,ORST,-7.0,61.0,ORE 34 - ORST 27


In [10]:
short_df.loc[mandatory_picks.index.tolist(), 'Mandatory'] = 'Y'
combined_df.loc[mandatory_picks.index.tolist(), 'Mandatory'] = 'Y'
short_df[short_df['Mandatory'] == 'Y']

,Week,Datetime,Mandatory,Favorite,Location,Underdog,Spread,Total,Implied Score
3,13,11/25 07:30 PM (Thu),Y,MSST,vs,MISS,-1.5,62.0,MSST 31 - MISS 30
4,13,11/26 12:00 PM (Fri),Y,BSU,@,SDSU,-2.5,44.5,BSU 23 - SDSU 21
12,13,11/26 03:30 PM (Fri),Y,ARK,vs,MIZ,-14.5,62.5,ARK 38 - MIZ 24
16,13,11/26 04:00 PM (Fri),Y,UTAH,vs,COLO,-23.5,52.5,UTAH 38 - COLO 14
18,13,11/26 07:00 PM (Fri),Y,NCST,vs,UNC,-6.0,62.5,NCST 34 - UNC 28
21,13,11/27 12:00 PM (Sat),Y,OSU,@,MICH,-8.0,64.5,OSU 36 - MICH 28
23,13,11/27 12:00 PM (Sat),Y,BAY,vs,TTU,-14.0,52.0,BAY 33 - TTU 19
42,13,11/27 03:30 PM (Sat),Y,ORE,vs,ORST,-7.0,61.0,ORE 34 - ORST 27
43,13,11/27 03:30 PM (Sat),Y,PSU,@,MSU,-1.0,51.5,PSU 26 - MSU 25
56,13,11/27 07:30 PM (Sat),Y,OKST,vs,OU,-4.0,49.5,OKST 26 - OU 22


In [11]:
update_google_sheet(short_df, combined_df)

05:17:20 INFO:Using credentials directory '/Users/hoke/dev/football/credentials'
05:17:20 INFO:Obtained credentails from existing 'token.json' file
05:17:20 DEBUG:Making request: POST https://oauth2.googleapis.com/token
05:17:20 DEBUG:Starting new HTTPS connection (1): oauth2.googleapis.com:443
05:17:20 DEBUG:https://oauth2.googleapis.com:443 "POST /token HTTP/1.1" 200 None
05:17:20 INFO:Refreshed expired credentials
05:17:20 INFO:Writing data for Hoke
05:17:20 INFO:file_cache is only supported with oauth2client<4.0.0
05:17:20 DEBUG:Attempting to write 65 rows to Hoke!A:J
05:17:20 DEBUG:URL being requested: POST https://sheets.googleapis.com/v4/spreadsheets/1tyso62sW52JGnJpFsHPSMNv0hnqUCpStgmKUuRVfcyg/values/Hoke%21A%3AJ:append?valueInputOption=USER_ENTERED&alt=json
05:17:20 INFO:Appended 585 cells to Hoke!A:J
05:17:20 INFO:Writing data for Mike
05:17:20 INFO:file_cache is only supported with oauth2client<4.0.0
05:17:20 DEBUG:Attempting to write 65 rows to Mike!A:J
05:17:21 DEBUG:URL b